## Exercício

In [2]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from scipy.stats import expon, reciprocal, uniform
from sklearn. svm import LinearSVR, SVR, NuSVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [3]:
housing = pd.read_csv('dataset/housing/housing.csv')

In [4]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
housing['income_cat'] = pd.cut(housing['median_income'],
                          bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                          labels=[1, 2, 3, 4, 5])

In [6]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42, shuffle=True, stratify=housing['income_cat'])

In [7]:
train_set = train_set.drop('income_cat', axis=1)
test_set = test_set.drop('income_cat', axis=1)

In [8]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

In [9]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, rooms_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [10]:
x_train = train_set.drop('median_house_value', axis=1)
y_train = train_set['median_house_value'].copy()

In [11]:
x_test = test_set.drop('median_house_value', axis=1)
y_test = test_set['median_house_value'].copy()

In [12]:
x_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN


In [13]:
numeric_attributes = list(x_train.drop('ocean_proximity', axis=1))
categorical_attributes = ['ocean_proximity']

In [14]:
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('combined_attributes', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

In [15]:
prepare_columns_pipeline = ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_attributes),
    ('categoric', OneHotEncoder(), categorical_attributes),
])

In [16]:
x = prepare_columns_pipeline.fit_transform(x_train)

### 1.

In [17]:
param_grid = [
    {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000.], 'epsilon': [0.1, 0.01]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300.], 'epsilon': [0.1, 0.01],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

In [18]:
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, verbose=2)

In [19]:
grid_search.fit(x, y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] END .................C=10.0, epsilon=0.1, kernel=linear; total time=   5.2s
[CV] END .................C=10.0, epsilon=0.1, kernel=linear; total time=   4.7s
[CV] END .................C=10.0, epsilon=0.1, kernel=linear; total time=   4.7s
[CV] END .................C=10.0, epsilon=0.1, kernel=linear; total time=   4.7s
[CV] END .................C=10.0, epsilon=0.1, kernel=linear; total time=   4.8s
[CV] END ................C=10.0, epsilon=0.01, kernel=linear; total time=   4.7s
[CV] END ................C=10.0, epsilon=0.01, kernel=linear; total time=   4.6s
[CV] END ................C=10.0, epsilon=0.01, kernel=linear; total time=   4.8s
[CV] END ................C=10.0, epsilon=0.01, kernel=linear; total time=   4.7s
[CV] END ................C=10.0, epsilon=0.01, kernel=linear; total time=   5.0s
[CV] END .................C=30.0, epsilon=0.1, kernel=linear; total time=   5.0s
[CV] END .................C=30.0, epsilon=0.1, 

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0],
                          'epsilon': [0.1, 0.01], 'kernel': ['linear']},
                         {'C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0],
                          'epsilon': [0.1, 0.01],
                          'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'kernel': ['rbf']}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [20]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

np.float64(70331.16627681501)

In [21]:
grid_search.best_params_

{'C': 3000.0, 'epsilon': 0.01, 'kernel': 'linear'}

{'C': 3000.0, 'epsilon': 0.01, 'kernel': 'linear'}

### 2.

In [22]:
param_distribs = {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'epsilon': uniform(0.01, 1),
        'gamma': expon(scale=1.0),
    }

In [23]:
svr = SVR()
random_search = RandomizedSearchCV(svr, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, random_state=42)

In [24]:
random_search.fit(x, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END C=629.7823295913721, epsilon=0.9607143064099162, gamma=1.3167456935454493, kernel=linear; total time=   5.8s
[CV] END C=629.7823295913721, epsilon=0.9607143064099162, gamma=1.3167456935454493, kernel=linear; total time=   5.7s
[CV] END C=629.7823295913721, epsilon=0.9607143064099162, gamma=1.3167456935454493, kernel=linear; total time=   5.6s
[CV] END C=629.7823295913721, epsilon=0.9607143064099162, gamma=1.3167456935454493, kernel=linear; total time=   5.7s
[CV] END C=629.7823295913721, epsilon=0.9607143064099162, gamma=1.3167456935454493, kernel=linear; total time=   5.6s
[CV] END C=4880.12141816351, epsilon=0.45583275285359115, gamma=0.10533264473289401, kernel=linear; total time=   6.7s
[CV] END C=4880.12141816351, epsilon=0.45583275285359115, gamma=0.10533264473289401, kernel=linear; total time=   6.4s
[CV] END C=4880.12141816351, epsilon=0.45583275285359115, gamma=0.10533264473289401, kernel=linear; total time

RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=50,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7be4c546f9b0>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7be4c5479670>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7be4c5454980>,
                                        'kernel': ['linear', 'rbf']},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [25]:
negative_mse = random_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

np.float64(58211.54062274842)

In [26]:
random_search.best_params_

{'C': np.float64(21012.398903194065),
 'epsilon': np.float64(0.4351558744912447),
 'gamma': np.float64(0.233120231885617),
 'kernel': 'rbf'}

### 3.

### 4.

In [ ]:
full_pipeline = Pipeline([
    ('preparation', prepare_columns_pipeline),
    ('svm_reg', SVR(**random_search.best_params_)),
])

In [31]:
model = full_pipeline.fit(x_train, y_train)

In [46]:
predicts = model.predict(x_test)
mse = mean_squared_error(y_test, predicts)
rmse = np.sqrt(mse)
rmse

np.float64(57428.90265891577)

### 5.